   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Wind Load Calculations\n",
    "\n",
    "### 3.1 Basic Wind Velocity (BS EN 1991-1-4)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Altitude factor calculation\n",
    "def calculate_altitude_factor(altitude):\n",
    "    \"\"\"Calculate altitude factor according to BS EN 1991-1-4\"\"\"\n",
    "    if altitude <= 10:\n",
    "        c_alt = 1 + 0.001 * altitude\n",
    "    else:\n",
    "        c_alt = 1 + 0.001 * altitude * (10/altitude)**0.2\n",
    "    return c_alt\n",
    "\n",
    "# Calculate altitude factor\n",
    "c_alt = calculate_altitude_factor(site_altitude)\n",
    "\n",
    "# Basic wind velocity\n",
    "vb_0 = vb_map * c_alt  # 10-minute mean basic wind velocity [m/s]\n",
    "\n",
    "# Basic velocity pressure\n",
    "qb = 0.5 * air_density * vb_0**2 / 1000  # [kN/m2]\n",
    "\n",
    "print(f\"Basic Wind Parameters:\")\n",
    "print(f\"Altitude factor: {c_alt:.3f}\")\n",
    "print(f\"Basic wind velocity: {vb_0:.2f} m/s\")\n",
    "print(f\"Basic velocity pressure: {qb:.3f} kN/m²\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 3.2 Exposure Factor & Peak Velocity Pressure"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Reference height (zs)\n",
    "zs = 0.6 * H + h0  # Reference height [m]\n",
    "\n",
    "# Exposure factor at reference height (simplified for terrain category II)\n",
    "# This is a simplified calculation - full implementation would need terrain category\n",
    "Ce_zs = 2.36  # From the original calculation\n",
    "\n",
    "# Peak velocity pressure at reference height\n",
    "qp_zs = Ce_zs * qb  # [kN/m2]\n",
    "\n",
    "# Peak wind velocity at reference height\n",
    "V_zs = sqrt(2 * qp_zs * 1000 / air_density)  # [m/s]\n",
    "\n",
    "print(f\"Reference Height Analysis:\")\n",
    "print(f\"Reference height zs: {zs:.1f} m\")\n",
    "print(f\"Exposure factor at zs: {Ce_zs:.2f}\")\n",
    "print(f\"Peak velocity pressure: {qp_zs:.3f} kN/m²\")\n",
    "print(f\"Peak wind velocity: {V_zs:.1f} m/s\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 3.3 Force Coefficient & Structural Factor"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Reynolds number at reference height\n",
    "Re_zs = V_zs * (D/1000) / kinematic_viscosity\n",
    "\n",
    "# Force coefficient Cf0 (from Figure 7.28 EN 1991-1-4)\n",
    "Cf0 = 0.796  # From original calculation\n",
    "\n",
    "# End-effect factor\n",
    "slenderness = H / (D/1000)  # Slenderness ratio\n",
    "phi = 1.0  # Solidity ratio\n",
    "\n",
    "if slenderness < 70:\n",
    "    psi_lambda = 0.6977  # From original calculation\n",
    "else:\n",
    "    psi_lambda = 1.0\n",
    "\n",
    "# Pressure coefficient at top\n",
    "Cf = Cf0 * psi_lambda\n",
    "\n",
    "# Structural factor CsCd (from Annex B)\n",
    "CsCd = 0.965  # From original calculation - this requires complex calculation\n",
    "\n",
    "print(f\"Force Coefficients:\")\n",
    "print(f\"Reynolds number: {Re_zs:.0f}\")\n",
    "print(f\"Slenderness ratio: {slenderness:.1f}\")\n",
    "print(f\"Force coefficient Cf0: {Cf0:.3f}\")\n",
    "print(f\"End-effect factor: {psi_lambda:.4f}\")\n",
    "print(f\"Pressure coefficient: {Cf:.3f}\")\n",
    "print(f\"Structural factor: {CsCd:.3f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 3.4 Wind Load Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create height segments based on original calculation\n",
    "height_segments = [\n",
    "    (0, 150), (150, 350), (350, 500), (500, 1200), (1200, 1800),\n",
    "    (1800, 2000), (2000, 3000), (3000, 3600), (3600, 3850), (3850, 4000),\n",
    "    (4000, 4800), (4800, 5000), (5000, 5400), (5400, 5500), (5500, 5800),\n",
    "    (5800, 6000), (6000, 6500), (6500, 6800), (6800, 7000), (7000, 7200),\n",
    "    (7200, 7600), (7600, 8000), (8000, 8500), (8500, 9000), (9000, 9500),\n",
    "    (9500, 10000), (10000, 11000), (11000, 12000), (12000, 13500), (13500, 13500)\n",
    "]\n",
    "\n",
    "def calculate_wind_load_distribution():\n",
    "    \"\"\"Calculate wind load distribution along chimney height\"\"\"\n",
    "    \n",
    "    results = []\n",
    "    \n",
    "    for i, (h_from, h_to) in enumerate(height_segments):\n",
    "        # Height for calculation (mid-height or reference height)\n",
    "        h_calc = (h_from + h_to) / 2 / 1000  # Convert to meters\n",
    "        \n",
    "        # Simplified exposure factor calculation (height-dependent)\n",
    "        # This is simplified - actual calculation is more complex\n",
    "        if h_calc < 3.0:\n",
    "            Ce_h = 2.36\n",
    "        else:\n",
    "            Ce_h = 2.36 * (h_calc / 8.1)**0.16  # Simplified power law\n",
    "        \n",
    "        # Peak velocity pressure at height\n",
    "        qp_h = Ce_h * qb\n",
    "        \n",
    "        # Structural factor varies with height (simplified)\n",
    "        CsCd_h = CsCd\n",
    "        \n",
    "        # Force coefficient (simplified - constant)\n",
    "        Cf_h = Cf\n",
    "        \n",
    "        # Wind load per unit length\n",
    "        fw_total = gamma_Q * CsCd_h * qp_h * (D/1000) * Cf_h  # [kN/m]\n",
    "        \n",
    "        results.append({\n",
    "            'Segment': i+1,\n",
    "            'From [mm]': h_from,\n",
    "            'To [mm]': h_to,\n",
    "            'Ce': Ce_h,\n",
    "            'qp [kN/m2]': qp_h,\n",
    "            'Cf': Cf_h,\n",
    "            'fw [kN/m]': fw_total\n",
    "        })\n",
    "    \n",
    "    return pd.DataFrame(results)\n",
    "\n",
    "# Calculate wind load distribution\n",
    "wind_loads = calculate_wind_load_distribution()\n",
    "\n",
    "# Display first few rows\n",
    "print(\"Wind Load Distribution (first 10 segments):\")\n",
    "display(wind_loads.head(10))\n",
    "\n",
    "print(f\"\\nMaximum wind load: {wind_loads['fw [kN/m]'].max():.3f} kN/m\")\n",
    "print(f\"Average wind load: {wind_loads['fw [kN/m]'].mean():.3f} kN/m\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 3.5 Wind Load Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create visualization of wind load distribution\n",
    "fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))\n",
    "\n",
    "# Height vs wind load\n",
    "heights = [(row['From [mm]'] + row['To [mm]']) / 2000 for _, row in wind_loads.iterrows()]\n",
    "ax1.plot(wind_loads['fw [kN/m]'], heights, 'b-o', linewidth=2, markersize=4)\n",
    "ax1.set_xlabel('Wind Load [kN/m]')\n",
    "ax1.set_ylabel('Height [m]')\n",
    "ax1.set_title('Wind Load Distribution')\n",
    "ax1.grid(True, alpha=0.3)\n",
    "\n",
    "# Peak velocity pressure distribution\n",
    "ax2.plot(wind_loads['qp [kN/m2]'], heights, 'r-o', linewidth=2, markersize=4)\n",
    "ax2.set_xlabel('Peak Velocity Pressure [kN/m²]')\n",
    "ax2.set_ylabel('Height [m]')\n",
    "ax2.set_title('Velocity Pressure Distribution')\n",
    "ax2.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Foundation Loads\n",
    "\n",
    "### 4.1 Load Integration"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def calculate_foundation_loads():\n",
    "    \"\"\"Calculate foundation reactions from wind and dead loads\"\"\"\n",
    "    \n",
    "    # Dead load (characteristic values)\n",
    "    N_dead = total_dead_load_kN  # [kN]\n",
    "    \n",
    "    # Initialize wind load integration\n",
    "    total_wind_force = 0  # [kN]\n",
    "    total_wind_moment = 0  # [kNm]\n",
    "    \n",
    "    # Integrate wind loads\n",
    "    for _, row in wind_loads.iterrows():\n",
    "        h_from = row['From [mm]'] / 1000  # [m]\n",
    "        h_to = row['To [mm]'] / 1000      # [m]\n",
    "        segment_length = h_to - h_from    # [m]\n",
    "        h_center = (h_from + h_to) / 2    # [m]\n",
    "        fw = row['fw [kN/m]']             # [kN/m]\n",
    "        \n",
    "        # Segment wind force\n",
    "        segment_force = fw * segment_length  # [kN]\n",
    "        total_wind_force += segment_force\n",
    "        \n",
    "        # Moment about base\n",
    "        total_wind_moment += segment_force * h_center  # [kNm]\n",
    "    \n",
    "    # Foundation reactions (characteristic values)\n",
    "    Q_base = total_wind_force     # Lateral force at base [kN]\n",
    "    M_base = total_wind_moment    # Bending moment at base [kNm]\n",
    "    N_base = N_dead               # Axial force [kN]\n",
    "    \n",
    "    return {\n",
    "        'N_dead': N_dead,\n",
    "        'Q_base': Q_base,\n",
    "        'M_base': M_base,\n",
    "        'N_base': N_base\n",
    "    }\n",
    "\n",
    "# Calculate foundation loads\n",
    "foundation_loads = calculate_foundation_loads()\n",
    "\n",
    "print(f\"Foundation Loads (Characteristic Values):\")\n",
    "print(f\"Dead load: {foundation_loads['N_dead']:.1f} kN\")\n",
    "print(f\"Lateral force at base: {foundation_loads['Q_base']:.1f} kN\")\n",
    "print(f\"Bending moment at base: {foundation_loads['M_base']:.1f} kNm\")\n",
    "print(f\"Total axial force: {foundation_loads['N_base']:.1f} kN\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Shell Stress Analysis\n",
    "\n",
    "### 5.1 Internal Forces"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def calculate_internal_forces():\n",
    "    \"\"\"Calculate internal forces along chimney height\"\"\"\n",
    "    \n",
    "    results = []\n",
    "    \n",
    "    for i, (h_from, h_to) in enumerate(height_segments):\n",
    "        h_from_m = h_from / 1000  # [m]\n",
    "        h_to_m = h_to / 1000      # [m]\n",
    "        h_calc = (h_from_m + h_to_m) / 2  # [m]\n",
    "        \n",
    "        # Calculate loads above this section\n",
    "        N_z = 0  # Normal force [kN]\n",
    "        Q_z = 0  # Shear force [kN]\n",
    "        M_z = 0  # Bending moment [kNm]\n",
    "        \n",
    "        # Integrate loads from top down\n",
    "        for j in range(i, len(height_segments)):\n",
    "            h_seg_from = height_segments[j][0] / 1000\n",
    "            h_seg_to = height_segments[j][1] / 1000\n",
    "            \n",
    "            if h_seg_to > h_calc:\n",
    "                seg_length = h_seg_to - max(h_seg_from, h_calc)\n",
    "                h_seg_center = (max(h_seg_from, h_calc) + h_seg_to) / 2\n",
    "                \n",
    "                # Dead load in segment\n",
    "                seg_dead_load = total_dead_load_kN * seg_length / H  # [kN]\n",
    "                N_z += seg_dead_load * gamma_Gk\n",
    "                \n",
    "                # Wind load in segment\n",
    "                fw_seg = wind_loads.iloc[j]['fw [kN/m]']  # Already includes safety factor\n",
    "                seg_wind_force = fw_seg * seg_length\n",
    "                Q_z += seg_wind_force\n",
    "                M_z += seg_wind_force * (h_seg_center - h_calc)\n",
    "        \n",
    "        # Calculate stresses\n",
    "        A_t = section['A_mm2']  # [mm2]\n",
    "        W_t = section['W_mm3']  # [mm3]\n",
    "        \n",
    "        # Second-order effects (simplified)\n",
    "        e2_over_8 = 0.0011  # From original calculation\n",
    "        M_second_order = M_z * (1 + e2_over_8)\n",
    "        \n",
    "        # Normal and bending stress\n",
    "        sigma_N = N_z / A_t * 1000  # [N/mm2]\n",
    "        sigma_M = M_second_order / W_t * 1e6  # [N/mm2]\n",
    "        sigma_total = sigma_N + sigma_M  # [N/mm2]\n",
    "        \n",
    "        # Shear stress\n",
    "        tau = Q_z / (2 * pi * section['rm'] * t_corroded) * 1000  # [N/mm2]\n",
    "        \n",
    "        results.append({\n",
    "            'Section': i+1,\n",
    "            'Height [m]': h_calc,\n",
    "            'N [kN]': N_z,\n",
    "            'Q [kN]': Q_z,\n",
    "            'M [kNm]': M_z,\n",
    "            'M\" [kNm]': M_second_order,\n",
    "            'σ [N/mm2]': sigma_total,\n",
    "            'τ [N/mm2]': tau\n",
    "        })\n",
    "    \n",
    "    return pd.DataFrame(results)\n",
    "\n",
    "# Calculate internal forces and stresses\n",
    "stress_analysis = calculate_internal_forces()\n",
    "\n",
    "print(\"Internal Forces and Stresses (first 10 sections):\")\n",
    "display(stress_analysis.head(10))\n",
    "\n",
    "print(f\"\\nMaximum stress: {stress_analysis['σ [N/mm2]'].max():.1f} N/mm²\")\n",
    "print(f\"Maximum shear: {stress_analysis['τ [N/mm2]'].max():.2f} N/mm²\")"
   ]
  }
 ],

# Static Calculation - Steel Chimney

**Project:** Northern Combustion Systems  
**Order No.:** 441  
**Revision:** 2  

This notebook implements the structural calculations for a steel chimney according to European standards (EN 1993-3-2, EN 1991-1-4, etc.).

## Table of Contents
1. [Input Parameters](#1-input-parameters)
2. [Safety Factors & Material Properties](#2-safety-factors--material-properties)
3. [Wind Load Calculations](#3-wind-load-calculations)
4. [Foundation Loads](#4-foundation-loads)
5. [Shell Stress Analysis](#5-shell-stress-analysis)
6. [Natural Frequency](#6-natural-frequency)
7. [Buckling Verification](#7-buckling-verification)
8. [Foundation Design](#8-foundation-design)
9. [Results Summary](#9-results-summary)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import pi, sqrt, log, sin, cos, tan
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Configure display options
pd.set_option('display.precision', 3)
plt.style.use('seaborn-v0_8')

## 1. Input Parameters

### 1.1 Main Dimensions

In [ ]:
# Main Dimensions
H = 13.50          # Chimney height [m]
D = 1422           # Outside shell diameter [mm]
h0 = 0.00          # Foundation level above ground [m]
Htot = H + h0      # Total height [m]

print(f"Main Dimensions:")
print(f"Chimney height: {H} m")
print(f"Outside diameter: {D} mm")
print(f"Foundation level: {h0} m")
print(f"Total height: {Htot} m")

### 1.2 Materials & Section Properties

In [ ]:
# Materials
steel_grade = "S235JR"      # Structural steel grade
anchor_grade = "8.8"        # Anchor bolt grade

# Shell thickness (including corrosion allowance)
t_shell = 8.0              # Shell thickness [mm]
corrosion_allowance = 0.35 # Internal corrosion allowance [mm]
t_corroded = t_shell - corrosion_allowance  # Corroded thickness [mm]

# Flue specifications
flue_internal_dia = 1100    # Internal diameter [mm]
flue_thickness = 2.0        # Flue thickness [mm]
flue_steel = "1.4301"       # Flue steel grade
insulation_thickness = 50   # Insulation thickness [mm]

print(f"Material Properties:")
print(f"Structural steel: {steel_grade}")
print(f"Shell thickness: {t_shell} mm (corroded: {t_corroded} mm)")
print(f"Flue: {flue_internal_dia} mm dia, {flue_thickness} mm thick, {flue_steel}")
print(f"Insulation: {insulation_thickness} mm")

### 1.3 Site Conditions & Wind Data

In [ ]:
# Site data
site_altitude = 200         # Site altitude above sea level [m]
distance_to_sea = 40        # Distance to sea [km]
surface_roughness = 0.0002  # Surface roughness [m]
axis_distance = 999.0       # Distance to nearest circular object [m]

# Wind parameters (BS EN 1991-1-4)
vb_map = 25.50             # Basic wind velocity from map [m/s]
air_density = 1.226        # Air density [kg/m3]
kinematic_viscosity = 1.50e-5  # Kinematic viscosity [m2/s]

# Design parameters
design_lifetime = 25        # Design lifetime [years]
design_temperature = 50     # Design temperature for shell [°C]

print(f"Site Conditions:")
print(f"Altitude: {site_altitude} m")
print(f"Distance to sea: {distance_to_sea} km")
print(f"Basic wind velocity: {vb_map} m/s")
print(f"Design temperature: {design_temperature} °C")

### 1.4 Foundation & Anchor Configuration

In [ ]:
# Foundation anchoring
n_bolts = 20              # Number of anchor bolts
bolt_size = "M20"         # Bolt size
bolt_circle_dia = 1602    # Bolt circle diameter [mm]
bolt_cross_section = 245  # Bolt cross-sectional area [mm2]
baseplate_thickness = 15  # Base plate thickness [mm]

# Gusset specifications
gusset_thickness = 8      # Gusset thickness [mm]
gusset_height = 400       # Gusset height [mm]

print(f"Foundation Configuration:")
print(f"Anchor bolts: {n_bolts} x {bolt_size}")
print(f"Bolt circle diameter: {bolt_circle_dia} mm")
print(f"Base plate thickness: {baseplate_thickness} mm")
print(f"Gusset: {gusset_thickness} mm thick, {gusset_height} mm high")

## 2. Safety Factors & Material Properties

### 2.1 Reliability Class & Safety Factors

In [ ]:
# Reliability class (EN 1993-3-2)
reliability_class = 2       # Class 2: Normal industrial chimneys
execution_class = "EXC3"    # Execution class

# Partial safety factors (EN 1993-3-2)
gamma_Q = 1.4              # Safety factor for wind load
gamma_Gk = 1.1             # Safety factor for dead load
gamma_F_f = 1.0            # Safety factor for vortex shedding loads
gamma_m = 1.0              # Safety factor for materials
gamma_M1 = 1.1             # Safety factor for buckling
gamma_m_b = 1.25           # Safety factor for bolts
gamma_mf = 1.15            # Safety factor for fatigue

print(f"Safety System:")
print(f"Reliability class: {reliability_class}")
print(f"Execution class: {execution_class}")
print(f"Wind load safety factor: {gamma_Q}")
print(f"Dead load safety factor: {gamma_Gk}")
print(f"Buckling safety factor: {gamma_M1}")

### 2.2 Material Properties

In [ ]:
# Material properties at design temperature
Ft = 1.0000               # Reducing factor at design temperature
E = 210000                # Modulus of elasticity [N/mm2]
nu = 0.3                  # Poisson's ratio

# Steel S235JR properties
fyk = 235                 # Characteristic yield strength [N/mm2]
fyd = fyk / gamma_m       # Design yield strength [N/mm2]

# Thickness-dependent yield strength
if t_shell <= 16:
    fyd_thick = 355.0 / gamma_m
elif t_shell <= 40:
    fyd_thick = 345.0 / gamma_m
else:
    fyd_thick = 335.0 / gamma_m

# Weld strengths
fyd_butt = fyd            # Butt weld strength = base material
fyd_fillet = 207.8        # Fillet weld strength [N/mm2]

# Anchor bolt properties (8.8 grade)
fyd_anchor = 576          # Anchor bolt design strength [N/mm2]

print(f"Material Strengths:")
print(f"Steel yield strength: {fyd:.1f} N/mm2")
print(f"Fillet weld strength: {fyd_fillet:.1f} N/mm2")
print(f"Anchor bolt strength: {fyd_anchor} N/mm2")
print(f"Modulus of elasticity: {E} N/mm2")

### 2.3 Cross-Section Properties

In [ ]:
# Calculate section properties
def calculate_section_properties(diameter, thickness):
    """Calculate cross-sectional properties for circular hollow section"""
    
    # Areas
    A = pi * diameter * thickness / 1000  # Cross-sectional area [mm2 -> m2 for some calcs]
    A_mm2 = pi * diameter * thickness     # Cross-sectional area [mm2]
    
    # Mean diameter and radius
    Dm = diameter - thickness            # Mean diameter [mm]
    rm = Dm / 2                          # Mean radius [mm]
    
    # Section modulus
    W = pi * Dm**2 * thickness / 4 / 1000  # Section modulus [mm3 -> m3]
    W_mm3 = pi * Dm**2 * thickness / 4     # Section modulus [mm3]
    
    # Second moment of area
    I = pi * Dm**3 * thickness / 8         # Second moment of area [mm4]
    
    return {
        'A_mm2': A_mm2,
        'Dm': Dm,
        'rm': rm,
        'W_mm3': W_mm3,
        'I_mm4': I
    }

# Calculate for corroded section
section = calculate_section_properties(D, t_corroded)

print(f"Cross-Section Properties (corroded):")
print(f"Cross-sectional area: {section['A_mm2']:.0f} mm²")
print(f"Mean diameter: {section['Dm']:.1f} mm")
print(f"Section modulus: {section['W_mm3']:.0f} mm³")
print(f"Second moment of area: {section['I_mm4']:.0f} mm⁴")

### 2.4 Weight Calculations

In [ ]:
# Material densities
rho_steel = 7850          # Steel density [kg/m3]
rho_insulation = 100      # Insulation density [kg/m3] (typical)

# Calculate shell weight
shell_volume = pi * D * t_shell * H / 1e9  # Shell volume [m3]
shell_weight = shell_volume * rho_steel    # Shell weight [kg]

# Calculate flue weight (approximate)
flue_volume = pi * flue_internal_dia * flue_thickness * H / 1e9  # Flue volume [m3]
flue_weight = flue_volume * rho_steel * 0.9  # Flue weight [kg] (factor for stainless steel)

# Calculate insulation weight
insulation_volume = pi * ((flue_internal_dia + 2*flue_thickness + 2*insulation_thickness)**2 - 
                         (flue_internal_dia + 2*flue_thickness)**2) * H / (4 * 1e9)
insulation_weight = insulation_volume * rho_insulation

# Total dead load
total_dead_load = shell_weight + flue_weight + insulation_weight  # [kg]
total_dead_load_kN = total_dead_load * 9.81 / 1000              # [kN]

print(f"Weight Summary:")
print(f"Shell weight: {shell_weight:.0f} kg")
print(f"Flue weight: {flue_weight:.0f} kg")
print(f"Insulation weight: {insulation_weight:.0f} kg")
print(f"Total dead load: {total_dead_load:.0f} kg ({total_dead_load_kN:.1f} kN)")

Now I'll mark the first task as completed and move to the next section: